In [1]:
#from skimage import io as skio
import glob
import pandas as pd
import os
import yaml
import shutil
from sklearn.model_selection import train_test_split


In [6]:
def train_from_anns(images_path, train_name):
        # might need a way to differentiate subsequent re-trainings
        # uuid = str(uuid.uuid4())[:2]
        # train_name = train_name + "_" + uuid

        # print("Train Name passed to training:")
        # print(train_name)

        #make training dirs
        path = r'C:\Users\gdea\OneDrive - Novozymes A S\Documents\python_files\Useful_work_Programs\ferm_eye_project\yolo_training' # output location 
        img_train = os.path.join(path, train_name, 'images/train')
        img_val = os.path.join(path, train_name, 'images/val')
        img_test = os.path.join(path, train_name, 'images/test')
        label_train = os.path.join(path, train_name, 'labels/train')
        label_val = os.path.join(path, train_name, 'labels/val')
        label_test = os.path.join(path, train_name, 'labels/test')

        folder_pth = os.path.join(path, train_name)

        os.makedirs(os.path.join(path, train_name), exist_ok=True)

        os.makedirs(os.path.join(path, train_name, 'images'), exist_ok=True)
        os.makedirs(os.path.join(path, train_name, 'labels'), exist_ok=True)
        os.makedirs(img_train, exist_ok=True)
        os.makedirs(img_val, exist_ok=True)
        os.makedirs(img_test, exist_ok=True)
        os.makedirs(label_train, exist_ok=True)
        os.makedirs(label_val, exist_ok=True)
        os.makedirs(label_test, exist_ok=True)

        #create YAML file

        # Needs to be changed depending on how many classes you have
        data = dict(
            train=os.path.join(path, train_name, 'images/train'), 
            val=os.path.join(path, train_name, 'images/val'),
            test=os.path.join(path, train_name, 'images/test'),
            nc=4, #number of classes
            names=["foam", "fermentation", "foam_cap", "rolling foam"], #class names - need to be in same order as they were trained 
        )
        with open(os.path.join(path, train_name, f'{train_name}.yaml'), 'w') as outfile:
            yaml.dump(data, outfile, default_flow_style=False)

        # get list of images and list of annotations_txts
        images = [os.path.join(images_path, x) for x in os.listdir(images_path) if x.endswith(".PNG")] # Change to extension of choice
        annotations = [os.path.join(images_path, x) for x in os.listdir(images_path) if x.endswith(".txt")]

        images.sort()
        annotations.sort()

        # Split the dataset into train-valid-test splits 
        train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
        val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

        def copy_files_to_folder(list_of_files, destination_folder):
            for f in list_of_files:
                try:
                    #should this be shutil.move?? or do we want them in both locations
                    shutil.copy(f, destination_folder)
                except:
                    print(f)
                    assert False

        # Move the splits into their folders
        copy_files_to_folder(train_images, img_train)
        copy_files_to_folder(val_images, img_val)
        copy_files_to_folder(test_images, img_test)
        copy_files_to_folder(train_annotations, label_train)
        copy_files_to_folder(val_annotations, label_val)
        copy_files_to_folder(test_annotations, label_test)

In [7]:
# Directory where you unzip stuff 
#
folder = r"C:\Users\gdea\OneDrive - Novozymes A S\Documents\python_files\Useful_work_Programs\ferm_eye_project\yolo_training\hu_58_first_pass_annotations_images\obj_train_data" #Once unzipped, this folder will point to the YAML file

new_model = "01092024_FermEye_hu58_firstpass" #Doesnt really matter, but should probably change

try:
    print("Attempting test/train split...")
    train_from_anns(folder, new_model)
    print("SUCCESS")
except Exception as e:
    print("FAILED")
    print(str(e))

Attempting test/train split...
SUCCESS
